<a href="https://colab.research.google.com/github/Cralsic123/PS4E5_flood/blob/main/PS4E5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install great_tables

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import warnings
import scipy
from sklearn.compose import TransformedTargetRegressor
from sklearn import set_config
from colorama import Style, Fore


In [ ]:
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
from sklearn.model_selection import StratifiedKFold, KFold
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, LinearRegression
from lightgbm import LGBMRegressor
from category_encoders import TargetEncoder, OneHotEncoder, MEstimateEncoder, OrdinalEncoder
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import roc_auc_score, roc_curve, make_scorer, mean_squared_log_error, r2_score
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import FunctionTransformer, StandardScaler, LabelEncoder, LabelBinarizer, MinMaxScaler, PolynomialFeatures, SplineTransformer
from sklearn.compose import ColumnTransformer
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform
from catboost import CatBoostRegressor
from sklearn.ensemble import HistGradientBoostingRegressor, VotingRegressor, RandomForestRegressor

In [ ]:
from great_tables import GT, style ,exibble, from_column, loc
from colorama import Style, Fore

In [ ]:
sns.set_theme(style = 'white', palette = 'colorblind')
pal = sns.color_palette('colorblind')



In [ ]:
pd.set_option('display.max_rows', 100)
set_config(transform_output = 'pandas')
pd.options.mode.chained_assignment = None
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
palette = ['#20B2AA', '#D8BFD8', '#8B0000', '#C09741',
           '#EC5B6D', '#90A6B1', '#6ca957', '#D8E3E2']

config = {
    'SEED' : 42,
    'N_SPLITS': 5,
    'SUBMIT' : True,
    'USE_ORIGINAL': False

}

# The Functions that are to be used

In [ ]:
def printColor(pText: str):
    print(f'{Style.BRIGHT}{Fore.GREEN}{pText}{Style.RESET_ALL}')

In [ ]:
def printInfo():
    print(f'{Style.BRIGHT}{Fore.YELLOW}SHAPE{Style.RESET_ALL}')
    print(f'{Style.BRIGHT}{Fore.GREEN} train: {train.shape}')
    print(f'{Style.BRIGHT}{Fore.GREEN} test:  {test.shape}')
    print(f'{Style.BRIGHT}{Fore.GREEN} original:  {original.shape}')
    print(f'{Style.BRIGHT}{Fore.YELLOW}\nNULL VALUES{Style.RESET_ALL}')
    print(f'{Style.BRIGHT}{Fore.GREEN} train: {train.isnull().any().any()}')
    print(f'{Style.BRIGHT}{Fore.GREEN} train: {test.isnull().any().any()}')
    print(f'{Style.BRIGHT}{Fore.GREEN} original: {original.isnull().any().any()}')
    print(f'{Style.BRIGHT}{Fore.YELLOW}\nDUPLICATES{Style.RESET_ALL}')
    print(f'{Style.BRIGHT}{Fore.GREEN} train: {train.duplicated().any().any()}')
    print(f'{Style.BRIGHT}{Fore.GREEN} train: {test.duplicated().any().any()}')
    print(f'{Style.BRIGHT}{Fore.GREEN} original: {original.duplicated().any().any()}')

In [ ]:
def customStatistic(df: pd.DataFrame(), categoric = False):
    num_cols = list(df._get_numeric_data())
    cat_cols = list(df.drop(num_cols,axis=1))
    if categoric:
        desc = pd.DataFrame(index = list(df[cat_cols]))
        df = df[cat_cols]
    else:
        desc = pd.DataFrame(index = list(df[num_cols]))
        df = df[num_cols]
        desc['skew'] = df[num_cols].skew()

    desc['type'] = df.dtypes
    desc['count'] = df.count()
    desc['nunique'] = df.nunique()
    desc['%unique'] = desc['nunique'] /len(df) * 100
    desc['null'] = df.isnull().sum()
    desc['%null'] = desc['null'] / len(df) * 100
    desc = pd.concat([desc,df.describe().T.drop('count',axis=1)],axis=1)

    desc = desc.round(2)
    return desc.reset_index().rename(columns={'index':'Column'}).sort_values(by=['type'])

In [ ]:
def min_max_unique(data_train, data_test):
     df = pd.DataFrame(index=data_train.columns)
    summary = {}
    for col in data_train.columns:
        if pd.api.types.is_numeric_dtype(data_train[col]):  # Verifica se a coluna é numérica
            min_train = min(data_train[col])
            min_test = min(data_test[col])
            max_train = max(data_train[col])
            max_test = max(data_test[col])
            unique_train = len(data_train[col].unique())
            unique_test = len(data_test[col].unique())
            top5_train = sorted(data_train[col])[:5]
            top5_test = sorted(data_test[col])[:5]
        else:
            min_train = min_test = max_train = max_test = None
            unique_train = len(data_train[col].unique())
            unique_test = len(data_test[col].unique())
            top5_train = top5_test = None
        summary[col] = [min_train, min_test, max_train, max_test,
                        unique_train, unique_test]

    df = pd.DataFrame.from_dict(summary, orient='index', columns=['min_train', 'min_test', 'max_train', 'max_test',
                                                                  'unique_train', 'unique_test'])\
        .reset_index().rename(columns={'index': 'columns'})


    return df
